In [1]:
from nftools import guis
import pylsl
import time
import numpy as np
import mne, re

%matplotlib qt  
import warnings; warnings.filterwarnings('ignore') 

### Prepare the EEG Cap

- fix the EEG Cap
- run the `python raw_eo_data --stream`, followed by `/start`
- (re)-start the Cap or USB if it doesn't work, followed by commands above

In [2]:
# make the connection to the EEG Net (EEG Data)
data_stream=pylsl.resolve_byprop("name", "openbci_eeg", timeout=5.0)
if data_stream:
    data_inlet=pylsl.stream_inlet(data_stream[0])
    stream_info = data_inlet.info()
    stream_Fs = stream_info.nominal_srate()
    stream_xml = stream_info.desc()
    chans_xml = stream_xml.child("channels")
    chan_xml_list = []
    ch = chans_xml.child("channel")
    while ch.name() == "channel":
        chan_xml_list.append(ch)
        ch = ch.next_sibling("channel")
    channel_names = [ch_xml.child_value("label") for ch_xml in chan_xml_list]
    data_inlet_dt = data_inlet.time_correction(timeout=5.0)
    sampling_freq = data_stream[0].nominal_srate()
    print('name = %s' % data_stream[0].name())
    print('sampling_freq = %d' % sampling_freq)
    print('channel_count = %d' % data_stream[0].channel_count())
    print('channel_format = %d' % data_stream[0].channel_format())
else:
    raise Exception('No Data Stream Found - Is your EEG Cap running?')

name = openbci_eeg
sampling_freq = 125
channel_count = 16
channel_format = 1


### Get (~ 2-3 minutes) of Eyes Open data

In [3]:
# clear EEG Data buffer
while data_inlet.samples_available(): data_inlet.pull_chunk() 

# collect some data    
np_eo = []
w=guis.AcquireData(sampling_freq, channel_names)
while w.RUNLOOP:
    chunk_data, chunk_times = data_inlet.pull_chunk() # grab from LSL
    if chunk_data:  # if there is any data
        np_eo.append(chunk_data) # add to our list
        w.update(chunk_data) # update the GUI window

# and we make the MNE data file from it        
raw_eo = mne.io.RawArray(np.concatenate(np_eo).T,
                        mne.create_info(channel_names, 
                                    sampling_freq, 
                                    'eeg', 
                                    None)
                       )


Creating RawArray with float64 data, n_channels=16, n_times=2089
    Range : 0 ... 2088 =      0.000 ...    16.704 secs
Ready.


### Check Bad Channels and Bad Time Intervals
- Use the MNE Interface to click channels (those are bad)
- Use also the MNE Interface ('a' button) to mark/drag
bad segments

In [4]:
# plot it
raw_eo.plot(scalings='auto');

In [5]:
# Analysis
# extracting the ch names and time information
bad_channels = raw_eo.info['bads']
bad_segments = [(a['onset'], a['duration']) for a in raw_eo.annotations if re.search('BAD', a['description'])]
print(bad_channels)
for s in bad_segments: print('%.2g, %.2g' % (s[0], sum(s)))

# convert to ch mask; and samples to keep/remove
bad_channel_indices = [i for i, ch in enumerate(raw_eo.info['ch_names']) 
                    if ch in bad_channels]
bad_channel_mask = [ch not in bad_channels for ch in channel_names]

[range(int(b * sampling_freq), int((b+d) * sampling_freq)) for b, d in bad_segments]

if bad_segments:
    bad_segment_samples = np.concatenate([range(int(b * sampling_freq), int((b+d) * sampling_freq)) for b, d in bad_segments])
else:
    bad_segment_samples=[]

print('channels to remove: %s' % bad_channel_indices)
print('channel mask: ' + repr(bad_channel_mask))
print('number of samples to remove: %s' % len(bad_segment_samples))

# remove the bad channels and bad samples, make new data matrices:
np_eo_forica = raw_eo.copy().get_data().T

np_eo_forica = np.delete(np_eo_forica, bad_segment_samples, axis=0)
np_eo_forica = np.delete(np_eo_forica, bad_channel_indices, axis=1)

print('raw original shape: %d, %d' % raw_eo.get_data().shape)
print('np new shape: %d, %d' % np_eo_forica.shape)

# # inspect the new data matrix in the viewer
# raw_eo_forica = mne.io.RawArray(np_eo_forica.T,
#                         mne.create_info([n for n in channel_names if n not in bad_channels], 
#                                     sampling_freq, 
#                                     'eeg', 
#                                     None)
#                        )

# raw_eo_forica.plot(scalings='auto');

from pynfb.protocols.ssd.topomap_selector_ica import ICADialog

ica_rejection, _, _, ica_unmixing_matrix, _, _ = ICADialog.get_rejection(
    np_eo_forica, 
    [n for n in channel_names if n not in bad_channels], 
    sampling_freq,
    decomposition=None
)

ica_rejection = ica_rejection.expand_by_mask(bad_channel_mask)

print('Created an ICA Spatial Filter')
print(ica_rejection)

['C3', 'T5', 'T6', 'C4']
7.7, 8.3
14, 16
channels to remove: [2, 3, 4, 5]
channel mask: [True, True, False, False, False, False, True, True, True, True, True, True, True, True, True, True]
number of samples to remove: 305
raw original shape: 16, 2089
np new shape: 1784, 12
apply filter: 3 to 45
Dropped 93 outliers
Creating RawArray with float64 data, n_channels=12, n_times=1691
    Range : 0 ... 1690 =      0.000 ...    13.520 secs
Ready.
Fitting ICA to data using 12 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Using all PCA components: 12
Computing Extended Infomax ICA
Fitting ICA took 1.2s.
ICA/CSP time elapsed = 1.212555170059204s
Table drawing time elapsed = 3.0800065994262695s
Created an ICA Spatial Filter


### Start up the Stimulus on Computer
- connect via WiFi
    - ip address = 10.42.0.1
    - host = stim-pc
    - password = PASSWORD

In [24]:
from callpyff import bcinetwork, bcixml
bcinet = bcinetwork.BciNetwork('10.42.0.1', bcinetwork.FC_PORT, bcinetwork.GUI_PORT, 'bcixml')

In [73]:
feedbacks = bcinet.getAvailableFeedbacks()
print(feedbacks)

['TestD2', 'MovingRhomb', 'EOEC', 'LibetClock', 'BrainWaveTraining_II', 'TobiQLAdapter', 'VisualOddballVE', 'EyetrackerFeedback', 'HexoSpeller', 'P300_Rectangle', 'ERPHex', 'BrainWaveTraining', 'StopVigilanceTask', 'FeedbackCursorArrow', 'TrivialPong', 'CheckerboardVEP', 'HexoSpellerVE', 'BoringClock', 'nback_verbal', 'Lesson01', 'BrainPong', 'CakeSpellerVE', 'MovingRhombGL', 'RestingState', 'NFBasicThermometer', 'RSVPSpeller', 'EEGfMRILocalizer', 'MultiVisualOddball', 'Lesson01b', 'GoalKeeper', 'CenterSpellerVE', 'Oddball', 'EyetrackerRawdata', 'StroopFeedback', 'ERPMatrix', 'Lesson04', 'Lesson05', 'Lesson06', 'VisualOddball', 'Lesson02', 'Lesson03']


In [74]:
bcinet.send_init('BrainWaveTraining_II')

In [75]:
bcinet.send_signal(bcixml.BciSignal({'EX_TESTNFNOISE': False},None, bcixml.INTERACTION_SIGNAL))
bcinet.send_signal(bcixml.BciSignal({'MONITOR_PIXWIDTH': 1366},None, bcixml.INTERACTION_SIGNAL))
bcinet.send_signal(bcixml.BciSignal({'MONITOR_PIXHEIGHT': 768},None, bcixml.INTERACTION_SIGNAL))
bcinet.send_signal(bcixml.BciSignal({'MONITOR_FULLSCR': True},None, bcixml.INTERACTION_SIGNAL))

In [25]:
# parameters to convert the filtered EEG signal to the stimulus
from nftools.nftools import signaltracking
track_for_eeg_stimuli = signaltracking.sending_to_nfstim(
    thr=1.0, 
    dur=0.20, 
    feedback_type='eeg', 
    max4audio=1.2, 
    bcinet=bcinet, 
    st_scaling=10
)

# parameters to convert the filtered EMG signal to the stimulus
track_for_emg_stimuli = signaltracking.sending_to_nfstim(
    thr=10, 
    dur=0.15, 
    feedback_type='emg', 
    bcinet=bcinet, 
    st_scaling=30
)

thr: 1.00, dur: 0.20
bcinet is passed on
thr: 10.00, dur: 0.15
bcinet is passed on


In [76]:
bcinet.play()

### Define Real-Time EEG Signal Analysis during NF

In [87]:
from pynfb.signal_processing.filters import (FilterSequence, 
                                             CFIRBandEnvelopeDetector, 
                                             ExponentialSmoother,
                                             SpatialFilter,
                                            )

In [ ]:
# Define Filter Sequence for NF

# Which channel do we select for the EEG?
rt_eeg_channels = ['C3']
rt_eeg_channels_mask = np.where([ch in rt_eeg_channels for ch in channel_names], 1, 0)/len(rt_eeg_channels)

rt_filters_eeg = [
    ica_rejection,
    SpatialFilter(rt_channels_mask),
    ButterFilter([1, None], sampling_freq, 1),
    CFIRBandEnvelopeDetector([12, 15], sampling_freq, ExponentialSmoother(0.9))
]


# which channel for the EMG (Muscle)?
rt_emg_channels = ['T3', 'T4'] # check if they're not in the bad channels..
rt_emg_channels_mask = np.where([ch in rt_eeg_channels for ch in channel_names], 1, 0)/len(rt_eeg_channels)

rt_filters_emg = [
    SpatialFilter(),
    ButterFilter([None, 50], sampling_freq, 1),
    CFIRBandEnvelopeDetector([None, 50], sampling_freq, ExponentialSmoother(0.9))
]


### Run the experiment
 - grab the data, apply RT Analysis, send to stimulus computer
 - press Enter on the Stimulus Computer to get started
 - subject trans (or should train) ther EEG

In [ ]:
# clear EEG Data buffer
while data_inlet.samples_available(): data_inlet.pull_chunk() 

# collect some data    
np_nf = []
w=guis.AcquireData(sampling_freq, channel_names)
while w.RUNLOOP:
    chunk_data, chunk_times = data_inlet.pull_chunk() # grab from LSL
    if chunk_data:  # if there is any data
        np_nf.append(chunk_data) # add to our list
        w.update(chunk_data) # update the GUI window
        
        s_eeg = rt_filters_eeg.apply(chunk_data)

        tf, audioTF = track_for_eeg_stimuli.check_above_threshold(s_eeg)  # sends markers (should be fast)
        s = track_for_eeg_stimuli.send_data_signal(s_eeg) # sends the signal (should also be fast!)

        
        s_emg = rt_filters_emg.apply(chunk_data)

        tfemg, audioTFemg = track_for_emg_stimuli.check_above_threshold(s_emg)  # sends markers (should be fast)
        semg = track_for_emg_stimuli.send_data_signal(s_emg) # sends the signal (should also be fast!)
        
        